<a href="https://colab.research.google.com/github/yeqinghuang516/transfer-learning-and-inference-acceleration/blob/master/ResNet_152_Testing_on_Cars_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook is RECOMMENDED to run on Google Colab

## Clone our Project Repo from Github

In [0]:
! git clone https://github.com/yeqinghuang516/transfer-learning-and-inference-acceleration.git
% cd transfer-learning-and-inference-acceleration

## Download Datasets

In [0]:
! wget http://imagenet.stanford.edu/internal/car196/car_ims.tgz
! wget http://imagenet.stanford.edu/internal/car196/cars_annos.mat
! tar -xvf car_ims.tgz
! tar -xvf cars_annos.mat

In [0]:
%matplotlib notebook
import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data as td
import torchvision as tv
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import scipy.io
import nntools as nt
import time
from download_weights import download_file_from_google_drive

In [0]:
class FullCarDataset(td.Dataset):
    def __init__(self, root_dir, image_size = (224, 224)):
        super(FullCarDataset, self).__init__()
        self.root_dir = root_dir
        self.image_size = image_size
        self.cars_annos = scipy.io.loadmat(root_dir + 'cars_annos.mat')
        self.annos = self.cars_annos['annotations']
        self.annos = np.transpose(self.annos)
        
    def __len__(self):
        return self.annos.shape[0]
    
    def __getitem__(self, idx):
        img_path = self.root_dir + self.annos[idx][0][0][0]
        bbox_x1 = self.annos[idx][0][1][0][0]
        bbox_y1 = self.annos[idx][0][2][0][0]
        bbox_x2 = self.annos[idx][0][3][0][0]
        bbox_y2 = self.annos[idx][0][4][0][0]
        d = self.annos[idx][0][5][0][0]
        d = int(d)
        img = Image.open(img_path).convert('RGB')
        img = img.crop([max(0 , bbox_x1 - 16), max(0, bbox_y1 - 16), min(img.size[0], bbox_x2 + 16), min(img.size[1], bbox_y2 + 16)])
        h, w = img.size
        dim_diff = np.abs(h - w)
        # (upper / left) padding and (lower / right) padding
        pad1, pad2 = dim_diff // 2, dim_diff - dim_diff // 2
        # Determine padding
        pad = (0, pad1, 0, pad2) if h >= w else (pad1, 0, pad2, 0)   # left, top, right and bottom
        # Add padding
        img = tv.transforms.functional.pad(img, pad,fill = 0, padding_mode = 'constant' )
        transform = tv.transforms.Compose([
            tv.transforms.Resize(self.image_size),
            tv.transforms.ToTensor(),
            tv.transforms.Normalize(mean = [0.5, 0.5 ,0.5], std = [0.5, 0.5 ,0.5]),
            ])
        
        x = transform(img)
        return x, (d - 1)
    
    def number_of_classes(self):
        classes = []
        for annotations in self.annos:
            classes.append(annotations[0][5][0][0])
        classes = np.array(classes)
        return classes.max()

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

## Download the Weight Stored in Google Drive

In [0]:
file_id = '1-0FLcfLlxW5AJMXFv_HmqLsKVyCzGxsU'
destination = '/content/transfer-learning-and-inference-acceleration/checkpoint.pth.tar'
download_file_from_google_drive(file_id, destination)
pretrained_weights = '/content/transfer-learning-and-inference-acceleration/checkpoint.pth.tar'

## Initiate Testing Dataset

In [0]:
dataset_root_dir = '/content/transfer-learning-and-inference-acceleration/'
fullDataset = FullCarDataset(dataset_root_dir)
testset = td.Subset(fullDataset, range(8143, len(fullDataset)))

## Define the Transferred Network

In [0]:
class NNClassifier(nt.NeuralNetwork):
    def __init__(self):
        super(NNClassifier, self).__init__()
        self.cross_entropy = nn.CrossEntropyLoss()
        
    def criterion(self, y, d):
        return self.cross_entropy(y, d)

In [0]:
class ResNet152Transfer(NNClassifier):
    
    def __init__(self, num_classes, fine_tuning=True):
        super(ResNet152Transfer, self).__init__()
        resnet152 = tv.models.resnet152(pretrained=False)
        for param in resnet152.parameters():
            param.requires_grad = fine_tuning
                
        self.model = resnet152
        self.model.fc = nn.Linear(2048, num_classes)
        
        
        
    def forward(self, x):
      y = self.model(x)
      return y

## Define the StatsManager for Recording the Loss and Accuracy

In [0]:
class ClassificationStatsManager(nt.StatsManager):
    def __init__(self):
        super(ClassificationStatsManager, self).__init__()
        
    def init(self):
        super(ClassificationStatsManager, self).init()
        self.running_accuracy_top1 = 0
        self.running_accuracy_top5 = 0
        
    def accumulate(self, loss, x, y, d):
        super(ClassificationStatsManager, self).accumulate(loss, x, y, d)
        _, l = torch.max(y, 1)
        _, l_top5 = torch.topk(y, 5)
        self.running_accuracy_top1 += torch.mean((l == d).float())
        self.running_accuracy_top5 += torch.mean((l_top5 == d.view(-1,1).repeat(1,5)).float()) * 5
        
        
    def summarize(self):
        loss = super(ClassificationStatsManager, self).summarize()
        accuracy_top1 = 100 * self.running_accuracy_top1 / self.number_update
        accuracy_top5 = 100 * self.running_accuracy_top5 / self.number_update
        return {'loss': loss, 'top 1 accuracy': accuracy_top1, 'top 5 accuracy': accuracy_top5}

In [0]:
def evaluate(net, val_set):
    """Evaluates the experiment, i.e., forward propagates the validation set
    through the network and returns the statistics computed by the stats
    manager.
    """
    stats_manager = ClassificationStatsManager()
    net.eval()
    val_loader = td.DataLoader(val_set, batch_size= 256, shuffle=False, drop_last=False, pin_memory=True)
    with torch.no_grad():
        for x, d in val_loader:
            x, d = x.to(net.device), d.to(net.device)
            y = net.forward(x)
            loss = net.criterion(y, d)
            stats_manager.accumulate(loss.item(), x, y, d)
            
    net.train()
    return stats_manager.summarize()

## Initiate the Network and Load Weights

In [0]:
net = ResNet152Transfer(196)
net = net.to(device)
checkpoint_path = '/content/transfer-learning-and-inference-acceleration/checkpoint.pth.tar'
checkpoint = torch.load(checkpoint_path,map_location= net.device)
net.load_state_dict(checkpoint['Net']);

## Conduct Evaluation on Testing Dataset

In [0]:
evaluate(net, testset)

## Plot a sample graph for a single prediction

In [0]:
% matplotlib inline
img_idx = np.random.randint(len(fullDataset))

x, d = fullDataset[img_idx]
x = x[None,:,:,:]
d = torch.tensor(d)

net.eval();

with torch.no_grad():
  x, d = x.to(net.device), d.to(net.device)
  y = net.forward(x)
  y = torch.exp(y) / torch.exp(y).sum()
  conf1 , label1 = torch.max(y, 1)
  conf5, label5 = torch.topk(y, 5)

  
  
x = torch.flatten(x, start_dim = 0, end_dim = 1)
x = x.to('cpu').numpy()
x = np.moveaxis(x, [0, 1, 2], [2, 0 ,1])
x = (x + 1) / 2
x[x > 1] = 1
x[x < 0] = 0
conf5, label5 = conf5.to('cpu').numpy(), label5.to('cpu').numpy()
conf5 = np.concatenate(conf5)
label5 = list(np.concatenate(label5))



fig = plt.figure(figsize = (18, 10) )
ax1 = fig.add_subplot(121)
ax1.imshow(x)
ax1.set_title('Predicted: %d, Actual: %d' %(label5[0], d), fontsize = 18)
ax2 = fig.add_subplot(122)
ax2.barh(range(5), conf5)
ax2.invert_yaxis()
ax2.set_yticklabels([0] + label5, fontsize = 18);
ax2.set_title('Top 5 Predictions', fontsize = 18)
ax2.set_ylabel('Predicted Class', fontsize = 18)
ax2.set_xlabel('Confidence', fontsize = 18)

for i, v in enumerate(conf5):
  ax2.text(v + 0.01, i, str(np.round(v, decimals = 4)), fontweight='bold', fontsize = 14)


net.train();